In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
from torch import nn
from torchvision.transforms import ToTensor

In [ ]:
flood_data = pd.read_csv("/content/datasets/kerala.csv")
flood_data['FLOODS'].replace(['YES','NO'],[1,0],inplace=True)
flood_data


,SUBDIVISION,YEAR,JAN,FEB,MAR,APR,MAY,JUN,JUL,AUG,SEP,OCT,NOV,DEC,ANNUAL RAINFALL,FLOODS
0,KERALA,1901,28.7,44.7,51.6,160.0,174.7,824.6,743.0,357.5,197.7,266.9,350.8,48.4,3248.6,1
1,KERALA,1902,6.7,2.6,57.3,83.9,134.5,390.9,1205.0,315.8,491.6,358.4,158.3,121.5,3326.6,1
2,KERALA,1903,3.2,18.6,3.1,83.6,249.7,558.6,1022.5,420.2,341.8,354.1,157.0,59.0,3271.2,1
3,KERALA,1904,23.7,3.0,32.2,71.5,235.7,1098.2,725.5,351.8,222.7,328.1,33.9,3.3,3129.7,1
4,KERALA,1905,1.2,22.3,9.4,105.9,263.3,850.2,520.5,293.6,217.2,383.5,74.4,0.2,2741.6,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113,KERALA,2014,4.6,10.3,17.9,95.7,251.0,454.4,677.8,733.9,298.8,355.5,99.5,47.2,3046.4,1
114,KERALA,2015,3.1,5.8,50.1,214.1,201.8,563.6,406.0,252.2,292.9,308.1,223.6,79.4,2600.6,0
115,KERALA,2016,2.4,3.8,35.9,143.0,186.4,522.2,412.3,325.5,173.2,225.9,125.4,23.6,2176.6,0
116,KERALA,2017,1.9,6.8,8.9,43.6,173.5,498.5,319.6,531.8,209.5,192.4,92.5,38.1,2117.1,0


,YEAR,JAN,FEB,MAR,APR,MAY,JUN,JUL,AUG,SEP,OCT,NOV,DEC
0,1901,28.7,44.7,51.6,160.0,174.7,824.6,743.0,357.5,197.7,266.9,350.8,48.4
1,1902,6.7,2.6,57.3,83.9,134.5,390.9,1205.0,315.8,491.6,358.4,158.3,121.5
2,1903,3.2,18.6,3.1,83.6,249.7,558.6,1022.5,420.2,341.8,354.1,157.0,59.0
3,1904,23.7,3.0,32.2,71.5,235.7,1098.2,725.5,351.8,222.7,328.1,33.9,3.3
4,1905,1.2,22.3,9.4,105.9,263.3,850.2,520.5,293.6,217.2,383.5,74.4,0.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...
113,2014,4.6,10.3,17.9,95.7,251.0,454.4,677.8,733.9,298.8,355.5,99.5,47.2
114,2015,3.1,5.8,50.1,214.1,201.8,563.6,406.0,252.2,292.9,308.1,223.6,79.4
115,2016,2.4,3.8,35.9,143.0,186.4,522.2,412.3,325.5,173.2,225.9,125.4,23.6
116,2017,1.9,6.8,8.9,43.6,173.5,498.5,319.6,531.8,209.5,192.4,92.5,38.1


In [ ]:
y = flood_data

In [ ]:
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
class FloodDataset(Dataset):
    def __init__(self, data_file, transform=None):
        df = pd.read_csv(data_file)

        # Split dataframe into data and labels
        self.data = df.drop(columns=['floods'])
        self.labels = df['floods']

    def __len__(self):
        # Return the size of the dataset
        return len(self.data)

    def __getitem__(self, index):
      item = self.data.iloc[index]
      label = self.labels[index]
      # Convert data and label to PyTorch tensors
      data_tensor = torch.tensor(item.values, dtype=torch.float32)
      label_tensor = torch.tensor(label, dtype=torch.long)
      # Apply transformations if available
      if self.transform:
        data_tensor = self.transform(data_tensor)

        return data_tensor, label_tensor



In [107]:
# Example using correct case
x = flood_data.iloc[:,1:14]

x

,YEAR,JAN,FEB,MAR,APR,MAY,JUN,JUL,AUG,SEP,OCT,NOV,DEC
0,1901,28.7,44.7,51.6,160.0,174.7,824.6,743.0,357.5,197.7,266.9,350.8,48.4
1,1902,6.7,2.6,57.3,83.9,134.5,390.9,1205.0,315.8,491.6,358.4,158.3,121.5
2,1903,3.2,18.6,3.1,83.6,249.7,558.6,1022.5,420.2,341.8,354.1,157.0,59.0
3,1904,23.7,3.0,32.2,71.5,235.7,1098.2,725.5,351.8,222.7,328.1,33.9,3.3
4,1905,1.2,22.3,9.4,105.9,263.3,850.2,520.5,293.6,217.2,383.5,74.4,0.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...
113,2014,4.6,10.3,17.9,95.7,251.0,454.4,677.8,733.9,298.8,355.5,99.5,47.2
114,2015,3.1,5.8,50.1,214.1,201.8,563.6,406.0,252.2,292.9,308.1,223.6,79.4
115,2016,2.4,3.8,35.9,143.0,186.4,522.2,412.3,325.5,173.2,225.9,125.4,23.6
116,2017,1.9,6.8,8.9,43.6,173.5,498.5,319.6,531.8,209.5,192.4,92.5,38.1


In [ ]:
y = flood_data.iloc[:,-1]
y

0      1
1      1
2      1
3      1
4      0
      ..
113    1
114    0
115    0
116    0
117    1
Name: FLOODS, Length: 118, dtype: int64

In [96]:
from sklearn.model_selection import train_test_split
x_train, x_test = train_test_split(x, test_size= 0.2, random_state=42)
y_train, y_test = train_test_split(y, test_size= 0.2, random_state=42)
x_train

44     2432.4
12     2610.8
15     2945.3
70     3076.8
107    2524.5
        ...  
106    3489.6
14     3024.5
92     2717.7
51     2334.8
102    2394.9
Name:  ANNUAL RAINFALL, Length: 94, dtype: float64

In [97]:
import torchvision.transforms as transforms
from torchvision import datasets

training_x = torch.tensor(x_train.values, dtype=torch.float32)
testing_x = torch.tensor(x_test.values, dtype=torch.float32)
training_y = torch.tensor(y_train.values, dtype=torch.float32)
testing_y = torch.tensor(y_test.values, dtype=torch.float32)

# train_data =FloodDataset(training_data,transform= transforms.ToTensor())

In [98]:
from torch.utils.data.dataloader import DataLoader
from torch.utils import data
#dataset chunk to use for each iteration
batchsize = 32

train_dataloader = DataLoader(training_x, batch_size= batchsize)
test_dataloader = DataLoader(testing_x, batch_size = batchsize)

In [ ]:
#check the device used for training the model
import torch
device =(
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"using {device} device")

In [99]:
import torch.nn as nn

output_size = 1

class Distra(nn.Module):
  def _init_(self, input_size, hidden_size, num_layers, output_size):
    super (Distra, self). init ()
    self.hidden_size = hidden_size
    self.num_layers = num_layers
    self.lstm = nn. LSTM(input_size, hidden_size, num_layers, batch_first=True)
    self.fc = nn. Linear (hidden_size, output_size)

    def forward (self, x):
      h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size)
      c0= torch.zeros(self.num_layers, x.size(0), self.hidden_size)
      out, = self.lstm(x, (h0, c0))
      out = self.fc (out[:, -1, ])
      return out

In [ ]:
input_size = 1
hidden_size = 64
num_layers = 2

model = Distra(input_size,hidden_size, num_layers, output_size )
loss_fn = nn.MSELoss()
#optimizer to adjust the weights and baises of the nueral network e.g
#stochastic gradient(SGD), Adam etc.
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)
#lr is the learning rate(alpha)


In [104]:
#training function
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        #Zero_grad  sens the gradient to zero after every iteration
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [101]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [106]:
epochs = 80
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    # test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------


ValueError: ignored

In [ ]:
#save model
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

In [ ]:
#load model
model = Distra()
model.load_state_dict(torch.load("model.pth"))

In [ ]:
#making predictions
model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
  predict = model(x)